In [48]:
import duckdb
import pandas as pd
import requests
import json
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression

from sklearn.decomposition import PCA
import umap   # $pip install umap-learn
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [49]:
df = pd.read_csv("../data/df_baseline.csv")
df

,price,carat,depth,table,L/W ratio,city,clarity,color,cut
0,505,0.32,63.0,57.0,0.993151,Kimberly,VS2,H,Very Good
1,2686,0.71,65.5,55.0,1.016275,Las Vegas,VS1,G,Fair
2,738,0.41,63.8,56.0,0.991525,Kimberly,SI1,D,Good
3,9057,1.52,61.2,57.0,1.008119,Tel Aviv,SI2,F,Ideal
4,557,0.31,61.8,54.2,0.990847,Zurich,VS1,H,Ideal
...,...,...,...,...,...,...,...,...,...
40433,1116,0.35,62.2,58.0,1.015730,Amsterdam,IF,G,Premium
40434,975,0.40,62.4,55.0,1.006369,Luxembourg,VS2,E,Ideal
40435,8572,1.57,61.1,61.0,1.004038,Tel Aviv,SI2,G,Premium
40436,5154,0.73,61.4,56.0,0.998279,Paris,VVS1,F,Ideal


In [50]:
## Categorical values

In [51]:
df_cat = df[['clarity','color','cut']]
df_cat

,clarity,color,cut
0,VS2,H,Very Good
1,VS1,G,Fair
2,SI1,D,Good
3,SI2,F,Ideal
4,VS1,H,Ideal
...,...,...,...
40433,IF,G,Premium
40434,VS2,E,Ideal
40435,SI2,G,Premium
40436,VVS1,F,Ideal


In [52]:
cat_cols = ['clarity','color','cut']
df_one_hot_encoding = pd.get_dummies(df[cat_cols], 
                                          columns=cat_cols, 
                                          drop_first=True)
df_one_hot_encoding

,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,color_E,color_F,color_G,color_H,color_I,color_J,cut_Good,cut_Ideal,cut_Premium,cut_Very Good
0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40433,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
40434,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0
40435,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0
40436,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0


In [53]:
encoded = df_one_hot_encoding.values

In [7]:
%%time
#pca = PCA().fit(encoded)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.86 µs


In [8]:
#pca.explained_variance_ratio_

In [9]:
#fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(13,8))
#plt.plot(np.cumsum(pca.explained_variance_ratio_))
#plt.xlabel('Number of Components')
#plt.ylabel('Variance (%)')
#plt.ylim([0,1])
#plt.title('Diamonds Dataset Explained Variance')
#plt.show()

In [10]:
#pca_optimum = PCA(n_components=23)
#encoded_pca_optimum = pca_optimum.fit_transform(encoded)
#df_one_hot_encoding = pd.DataFrame(encoded_pca_optimum)
#df_one_hot_encoding

In [11]:
# Numerical Variables

In [54]:
df_num = df[['price','carat','depth','table','L/W ratio']]
df_num

,price,carat,depth,table,L/W ratio
0,505,0.32,63.0,57.0,0.993151
1,2686,0.71,65.5,55.0,1.016275
2,738,0.41,63.8,56.0,0.991525
3,9057,1.52,61.2,57.0,1.008119
4,557,0.31,61.8,54.2,0.990847
...,...,...,...,...,...
40433,1116,0.35,62.2,58.0,1.015730
40434,975,0.40,62.4,55.0,1.006369
40435,8572,1.57,61.1,61.0,1.004038
40436,5154,0.73,61.4,56.0,0.998279


In [13]:
# Concat Numerical and Categorical variables

In [55]:
df_concatenated = pd.concat([df_one_hot_encoding, df_num], axis=1)

In [56]:
df_concatenated

,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,color_E,color_F,color_G,...,color_J,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,price,carat,depth,table,L/W ratio
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,505,0.32,63.0,57.0,0.993151
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,2686,0.71,65.5,55.0,1.016275
2,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,738,0.41,63.8,56.0,0.991525
3,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,9057,1.52,61.2,57.0,1.008119
4,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,557,0.31,61.8,54.2,0.990847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40433,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,1116,0.35,62.2,58.0,1.015730
40434,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,975,0.40,62.4,55.0,1.006369
40435,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,8572,1.57,61.1,61.0,1.004038
40436,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,5154,0.73,61.4,56.0,0.998279


In [57]:
# Scaling
df_concatenated.columns = df_concatenated.columns.astype(str)
df_concatenated

,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,color_E,color_F,color_G,...,color_J,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,price,carat,depth,table,L/W ratio
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,505,0.32,63.0,57.0,0.993151
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,2686,0.71,65.5,55.0,1.016275
2,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,738,0.41,63.8,56.0,0.991525
3,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,9057,1.52,61.2,57.0,1.008119
4,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,557,0.31,61.8,54.2,0.990847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40433,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,1116,0.35,62.2,58.0,1.015730
40434,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,975,0.40,62.4,55.0,1.006369
40435,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,8572,1.57,61.1,61.0,1.004038
40436,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,5154,0.73,61.4,56.0,0.998279


In [58]:
df_price = df_concatenated[['price']]
df_price

,price
0,505
1,2686
2,738
3,9057
4,557
...,...
40433,1116
40434,975
40435,8572
40436,5154


In [59]:
df_concatenated.columns

Index(['clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'color_E', 'color_F',
       'color_G', 'color_H', 'color_I', 'color_J', 'cut_Good', 'cut_Ideal',
       'cut_Premium', 'cut_Very Good', 'price', 'carat', 'depth', 'table',
       'L/W ratio'],
      dtype='object')

In [61]:
df_concatenated1 = df_concatenated[['clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'color_E', 'color_F',
       'color_G', 'color_H', 'color_I', 'color_J', 'cut_Good', 'cut_Ideal',
       'cut_Premium', 'cut_Very Good', 'carat', 'depth', 'table',
       'L/W ratio']]

In [62]:
# Using scikit-learn .MinMaxScaler()

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_concatenated1)
scaled_data

array([[0.        , 0.        , 0.        , ..., 0.55555556, 0.26923077,
        0.28160271],
       [0.        , 0.        , 0.        , ..., 0.625     , 0.23076923,
        0.3082926 ],
       [0.        , 1.        , 0.        , ..., 0.57777778, 0.25      ,
        0.27972684],
       ...,
       [0.        , 0.        , 1.        , ..., 0.50277778, 0.34615385,
        0.29416847],
       [0.        , 0.        , 0.        , ..., 0.51111111, 0.25      ,
        0.28752161],
       [0.        , 0.        , 0.        , ..., 0.51944444, 0.26923077,
        0.2821566 ]])

In [63]:
scaled_df = pd.DataFrame(scaled_data)
scaled_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
count,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,...,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000,40438.000000
mean,0.032816,0.241036,0.171101,0.149983,0.228646,0.068599,0.093946,0.181142,0.177383,0.208937,...,0.099683,0.052921,0.090484,0.401058,0.253474,0.224912,0.138929,0.520919,0.277799,0.288833
std,0.178156,0.427717,0.376602,0.357059,0.419966,0.252774,0.291758,0.385140,0.381997,0.406554,...,0.299581,0.223878,0.286878,0.490119,0.435006,0.417530,0.110518,0.039757,0.042944,0.011956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046512,0.500000,0.250000,0.280974
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.116279,0.522222,0.269231,0.284586
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.195349,0.541667,0.307692,0.297505
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [64]:
result_df = pd.concat([scaled_df, df_price], axis=1)
result_df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,price
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.027907,0.555556,0.269231,0.281603,505
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.118605,0.625000,0.230769,0.308293,2686
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.048837,0.577778,0.250000,0.279727,738
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.306977,0.505556,0.269231,0.298879,9057
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.025581,0.522222,0.215385,0.278943,557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40433,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.034884,0.533333,0.288462,0.307664,1116
40434,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.046512,0.538889,0.230769,0.296860,975
40435,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.318605,0.502778,0.346154,0.294168,8572
40436,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.123256,0.511111,0.250000,0.287522,5154


In [65]:
result_df.to_csv("../data/training_datasets/one_hot_minmax.csv")

In [26]:
test_df = pd.read_csv("../data/tests/diamonds_test.csv")
test_df

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [28]:
cat_cols = ['city','clarity','color','cut']
test_one_hot_encoding = pd.get_dummies(test_df[cat_cols], 
                                          columns=cat_cols, 
                                          drop_first=True)
test_one_hot_encoding

,city_Antwerp,city_Dubai,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,...,color_E,color_F,color_G,color_H,color_I,color_J,cut_Good,cut_Ideal,cut_Premium,cut_Very Good
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
13481,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
13482,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
13483,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [40]:
test_df = pd.read_csv("../data/tests/test_baseline.csv")
test_df

,carat,depth,table,L/W ratio,city,clarity,color,cut
0,0.79,62.7,60.0,0.988115,Amsterdam,SI1,F,Very Good
1,1.20,61.0,57.0,0.988389,Surat,VS1,J,Ideal
2,1.57,62.2,61.0,1.008197,Kimberly,SI1,H,Premium
3,0.90,63.8,54.0,0.993475,Kimberly,SI1,F,Very Good
4,0.50,62.9,58.0,0.992141,Amsterdam,VS1,F,Very Good
...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,1.005639,Amsterdam,SI1,E,Ideal
13481,0.71,62.2,55.0,0.996510,New York City,VS2,I,Ideal
13482,0.70,61.6,55.0,1.007005,Tel Aviv,VS1,F,Ideal
13483,0.70,58.8,57.0,0.993209,Surat,SI2,F,Very Good


In [41]:
test_cat = test_df[['city','clarity','color','cut']]
test_cat

,city,clarity,color,cut
0,Amsterdam,SI1,F,Very Good
1,Surat,VS1,J,Ideal
2,Kimberly,SI1,H,Premium
3,Kimberly,SI1,F,Very Good
4,Amsterdam,VS1,F,Very Good
...,...,...,...,...
13480,Amsterdam,SI1,E,Ideal
13481,New York City,VS2,I,Ideal
13482,Tel Aviv,VS1,F,Ideal
13483,Surat,SI2,F,Very Good


In [42]:
cat_cols = ['city','clarity','color','cut']
test_one_hot_encoding = pd.get_dummies(test_df[cat_cols], 
                                          columns=cat_cols, 
                                          drop_first=True)
test_one_hot_encoding

,city_Antwerp,city_Dubai,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,...,color_E,color_F,color_G,color_H,color_I,color_J,cut_Good,cut_Ideal,cut_Premium,cut_Very Good
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
13481,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
13482,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
13483,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [43]:
test_num = test_df[['carat','depth','table','L/W ratio']]
test_num

,carat,depth,table,L/W ratio
0,0.79,62.7,60.0,0.988115
1,1.20,61.0,57.0,0.988389
2,1.57,62.2,61.0,1.008197
3,0.90,63.8,54.0,0.993475
4,0.50,62.9,58.0,0.992141
...,...,...,...,...
13480,0.57,61.9,56.0,1.005639
13481,0.71,62.2,55.0,0.996510
13482,0.70,61.6,55.0,1.007005
13483,0.70,58.8,57.0,0.993209


In [44]:
test_concatenated = pd.concat([test_one_hot_encoding, test_num], axis=1)
test_concatenated

,city_Antwerp,city_Dubai,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,...,color_I,color_J,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,carat,depth,table,L/W ratio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.79,62.7,60.0,0.988115
1,0,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,1.20,61.0,57.0,0.988389
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1.57,62.2,61.0,1.008197
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.90,63.8,54.0,0.993475
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.50,62.9,58.0,0.992141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0.57,61.9,56.0,1.005639
13481,0,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0.71,62.2,55.0,0.996510
13482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0.70,61.6,55.0,1.007005
13483,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0.70,58.8,57.0,0.993209


In [45]:
test_scaled = scaler.fit_transform(test_concatenated)

In [46]:
scaled_test = pd.DataFrame(test_scaled)
scaled_test

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.122661,0.421986,0.409091,0.573939
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.207900,0.361702,0.272727,0.574129
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.284823,0.404255,0.454545,0.587890
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.145530,0.460993,0.136364,0.577662
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.062370,0.429078,0.318182,0.576736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.076923,0.393617,0.227273,0.586113
13481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.106029,0.404255,0.181818,0.579771
13482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.103950,0.382979,0.181818,0.587062
13483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.103950,0.283688,0.272727,0.577478


In [47]:
scaled_test.to_csv("../data/tests/one_hot_min_max_test.csv")